In [25]:
import json
import pandas as pd

from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.document import Document
from langchain_community.vectorstores import FAISS
from pathlib import Path
from dotenv import load_dotenv; load_dotenv()

True

In [26]:
embedding_model = HuggingFaceEmbeddings(model_name = 'sentence-transformers/all-MiniLM-L6-v2')
embedding_model

HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

### Create Eval Dataframe and Corpus

In [40]:
file_path = '/Users/ton_kkrongyuth/Senior Project/Aj.Pannapa/Prove_LLM/Data/NATURALPROOFS_DATASET/naturalproofs_proofwiki.json'
raw_data = json.loads(Path(file_path).read_text())

corpus = []
eval_df = pd.DataFrame(columns=['throrem_titles',
                                'theorem_contents',
                                'theorem_refs',         # Actual 'reference title' from grounds truth
                                'ref_ids',              # Actual 'references ID' from grounds truth
                                'retrieved_3',          # Top 3 retrieved documents ID and L2 score from DB
                                'retrieved_5',          # Top 5 retrieved documents ID and L2 score from DB
                                'retrieved_10',         # Top 10 retrieved documents ID and L2 score from DB
                                'retrieved_50',         # Top 50 retrieved documents ID and L2 score from DB
                                'P@3',                  # Precision @ 3
                                'P@5',                  # Precision @ 5
                                'P@10',                 # Precision @ 10
                                'P@50',                 # Precision @ 50
                                'R@3',                  # Recall @ 3
                                'R@5',                  # Recall @ 5
                                'R@10',                 # Recall @ 10
                                'R@50'                  # Recall @ 50
                                ]) 

for _, category in enumerate(raw_data['dataset']):
    if category not in ['theorems', 'retrieval_examples']:
        for _, data in enumerate(raw_data['dataset'][category]):
            id = data['id']
            title = data['title']
            content = ''.join(data['contents'])
            
            document = f"""{title}: {content}"""
            corpus.append(Document(
                page_content = document,
                metadata = {'source': category, 'id': id}
                ))
    elif category == 'theorems':
        title, contents, refs, ref_ids = [], [], [], []
        for _, theorem in enumerate(raw_data['dataset'][category]):
            if theorem['ref_ids']:
                title.append(theorem['title'])
                contents.append(''.join(theorem['contents']))
                refs.append(theorem['refs'])
                ref_ids.append(theorem['ref_ids'])
        
        temp_df = pd.concat([eval_df,
                            pd.DataFrame({
                                'throrem_titles': title,
                                'theorem_contents': contents,
                                'theorem_refs': refs,
                                'ref_ids': ref_ids})],
                            ignore_index=True)
        eval_df = temp_df

eval_df

,throrem_titles,theorem_contents,theorem_refs,ref_ids,retrieved_3,retrieved_5,retrieved_10,retrieved_50,P@3,P@5,P@10,P@50,R@3,R@5,R@10,R@50
0,Closed Form for Triangular Numbers,The [[Definition:Closed-Form Expression|closed...,"[Definition:Closed-Form Expression, Definition...","[20933, 20514]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Union is Associative,[[Definition:Set Union|Set union]] is [[Defini...,"[Definition:Set Union, Definition:Associative ...","[19740, 25744]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pythagoras's Theorem,Let $\triangle ABC$ be a [[Definition:Right Tr...,"[Definition:Triangle (Geometry)/Right-Angled, ...","[20891, 24083]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Euclid's Theorem,For any [[Definition:Finite Set|finite set]] o...,"[Definition:Finite Set, Definition:Prime Numbe...","[21762, 20069, 20069, 19734]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Limsup Squeeze Theorem,Let $\left \langle {x_n} \right \rangle$ and $...,"[Definition:Real Sequence, Definition:Limit Su...","[21123, 20243]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13721,Direct Product of Banach Spaces is Banach Space,"Let $\struct {X, \norm {\, \cdot \,}}$ and $\s...","[Definition:Normed Vector Space, Definition:Di...","[21721, 27428, 22539, 32137, 21722, 21722]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13722,Cartesian Plane Rotated with respect to Another,Let $\mathbf r$ be a [[Definition:Position Vec...,"[Definition:Displacement, Definition:Cartesian...","[21291, 24208, 23177, 29628, 28368, 20152, 296...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13723,Dot Product of Perpendicular Vectors,Let $\mathbf a$ and $\mathbf b$ be [[Definitio...,"[Definition:Vector Quantity, Definition:Right ...","[20142, 24250, 20452]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13724,Dot Product of Orthogonal Basis Vectors,"Let $\tuple {\mathbf e_1, \mathbf e_2, \ldots,...","[Definition:Orthonormal Basis of Vector Space,...","[32145, 20141, 20452, 20133]",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
corpus, len(corpus)

([Document(metadata={'source': 'definitions', 'id': 19734}, page_content="Definition:Set: A '''set''' is intuitively defined as any aggregation of [[Definition:Object|objects]], called [[Definition:Element|elements]], which can be precisely defined in some way or other.We can think of each set as a single entity in itself, and we can denote it (and usually do) by means of a single symbol.That is, ''anything you care to think of'' can be a set. This concept is known as the [[Axiom:Comprehension Principle|comprehension principle]].However, there are problems with the [[Axiom:Comprehension Principle|comprehension principle]]. If we allow it to be used without any restrictions at all, paradoxes arise, the most famous example probably being [[Russell's Paradox]].Hence some sources define a '''set''' as a ''' 'well-defined' collection of [[Definition:Object|objects]]''', leaving the concept of what constitutes well-definition to later in the exposition."),
  Document(metadata={'source': 'def

### Create & Save retriever

In [10]:
refs_db = FAISS.from_documents(corpus, embedding_model)
refs_db.save_local(folder_path='./ref_db',
                   index_name='ref_db')

# Use as retriever
# refs_retriever = refs_db.as_retriever()
# refs_retriever

### Evaluate retrieve performance

In [41]:
refs_db = FAISS.load_local(
    folder_path='./ref_db',
    embeddings=embedding_model,
    index_name='ref_db',
    allow_dangerous_deserialization=True)
refs_db

In [42]:
def evaluate(eval_df:pd.DataFrame, retriever:FAISS):
    for _, data in enumerate(eval_df.iterrows()):
        theorem_query = data[1]['theorem_contents']
        actual_refs = set(data[1]['ref_ids'])
        
        retrieved_docs = retriever.similarity_search_with_score(
            query = theorem_query,
            k = 50
            )
        
        for k in [3, 5, 10, 50]:
            result_list = {doc[0].metadata['id']:doc[1] for doc in retrieved_docs[:k]}
            
            retrieve_ele = set(result_list.keys())
            retrieve_all = set({doc[0].metadata['id']:doc[1] for doc in retrieved_docs[:50]}.keys())
            intersect_k = actual_refs.intersection(retrieve_ele)
            intersect_all = actual_refs.intersection(retrieve_all)
            
            precision_K = len(intersect_k)/k
            if intersect_all:
                recall_K = len(intersect_k)/len(intersect_all)
                eval_df.at[data[0], f'R@{k}'] = recall_K
            else:
                """
                This condition for
                1.Retriever not retrieved any correct documents
                2.Theorem don't have any refference
                """
                
            eval_df.at[data[0], f'retrieved_{k}'] = result_list
            eval_df.at[data[0], f'P@{k}'] = precision_K
            
    eval_df.to_csv('retrieve_performance.csv')
        
    return eval_df

eval_df = evaluate(eval_df, refs_db)

In [43]:
eval_df

,throrem_titles,theorem_contents,theorem_refs,ref_ids,retrieved_3,retrieved_5,retrieved_10,retrieved_50,P@3,P@5,P@10,P@50,R@3,R@5,R@10,R@50
0,Closed Form for Triangular Numbers,The [[Definition:Closed-Form Expression|closed...,"[Definition:Closed-Form Expression, Definition...","[20933, 20514]","{26531: 0.42419058, 26532: 0.531752, 26533: 0....","{26531: 0.42419058, 26532: 0.531752, 26533: 0....","{26531: 0.42419058, 26532: 0.531752, 26533: 0....","{26531: 0.42419058, 26532: 0.531752, 26533: 0....",0.0,0.2,0.2,0.04,0.0,0.5,1.0,1.0
1,Union is Associative,[[Definition:Set Union|Set union]] is [[Defini...,"[Definition:Set Union, Definition:Associative ...","[19740, 25744]","{25744: 0.69588184, 25960: 0.8464983, 19740: 0...","{25744: 0.69588184, 25960: 0.8464983, 19740: 0...","{25744: 0.69588184, 25960: 0.8464983, 19740: 0...","{25744: 0.69588184, 25960: 0.8464983, 19740: 0...",0.666667,0.4,0.2,0.04,1.0,1.0,1.0,1.0
2,Pythagoras's Theorem,Let $\triangle ABC$ be a [[Definition:Right Tr...,"[Definition:Triangle (Geometry)/Right-Angled, ...","[20891, 24083]","{27326: 0.82935846, 20891: 0.88775295, 24083: ...","{27326: 0.82935846, 20891: 0.88775295, 24083: ...","{27326: 0.82935846, 20891: 0.88775295, 24083: ...","{27326: 0.82935846, 20891: 0.88775295, 24083: ...",0.666667,0.4,0.2,0.04,1.0,1.0,1.0,1.0
3,Euclid's Theorem,For any [[Definition:Finite Set|finite set]] o...,"[Definition:Finite Set, Definition:Prime Numbe...","[21762, 20069, 20069, 19734]","{27451: 0.78793746, 20069: 0.8540012, 27455: 0...","{27451: 0.78793746, 20069: 0.8540012, 27455: 0...","{27451: 0.78793746, 20069: 0.8540012, 27455: 0...","{27451: 0.78793746, 20069: 0.8540012, 27455: 0...",0.333333,0.2,0.2,0.04,0.5,0.5,1.0,1.0
4,Limsup Squeeze Theorem,Let $\left \langle {x_n} \right \rangle$ and $...,"[Definition:Real Sequence, Definition:Limit Su...","[21123, 20243]","{29488: 0.5679671, 30170: 0.6801356, 29489: 0....","{29488: 0.5679671, 30170: 0.6801356, 29489: 0....","{29488: 0.5679671, 30170: 0.6801356, 29489: 0....","{29488: 0.5679671, 30170: 0.6801356, 29489: 0....",0.0,0.2,0.1,0.02,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13721,Direct Product of Banach Spaces is Banach Space,"Let $\struct {X, \norm {\, \cdot \,}}$ and $\s...","[Definition:Normed Vector Space, Definition:Di...","[21721, 27428, 22539, 32137, 21722, 21722]","{32137: 0.30584806, 31307: 0.7789068, 31891: 0...","{32137: 0.30584806, 31307: 0.7789068, 31891: 0...","{32137: 0.30584806, 31307: 0.7789068, 31891: 0...","{32137: 0.30584806, 31307: 0.7789068, 31891: 0...",0.333333,0.6,0.3,0.08,0.25,0.75,0.75,1.0
13722,Cartesian Plane Rotated with respect to Another,Let $\mathbf r$ be a [[Definition:Position Vec...,"[Definition:Displacement, Definition:Cartesian...","[21291, 24208, 23177, 29628, 28368, 20152, 296...","{29628: 0.79528964, 32126: 0.83278644, 32127: ...","{29628: 0.79528964, 32126: 0.83278644, 32127: ...","{29628: 0.79528964, 32126: 0.83278644, 32127: ...","{29628: 0.79528964, 32126: 0.83278644, 32127: ...",0.333333,0.2,0.1,0.1,0.2,0.2,0.2,1.0
13723,Dot Product of Perpendicular Vectors,Let $\mathbf a$ and $\mathbf b$ be [[Definitio...,"[Definition:Vector Quantity, Definition:Right ...","[20142, 24250, 20452]","{24752: 0.6457876, 24860: 0.6728038, 22482: 0....","{24752: 0.6457876, 24860: 0.6728038, 22482: 0....","{24752: 0.6457876, 24860: 0.6728038, 22482: 0....","{24752: 0.6457876, 24860: 0.6728038, 22482: 0....",0.0,0.0,0.0,0.04,0.0,0.0,0.0,1.0
13724,Dot Product of Orthogonal Basis Vectors,"Let $\tuple {\mathbf e_1, \mathbf e_2, \ldots,...","[Definition:Orthonormal Basis of Vector Space,...","[32145, 20141, 20452, 20133]","{20452: 0.54099584, 32145: 0.60490584, 32144: ...","{20452: 0.54099584, 32145: 0.60490584, 32144: ...","{20452: 0.54099584, 32145: 0.60490584, 32144: ...","{20452: 0.54099584, 32145: 0.60490584, 32144: ...",0.666667,0.4,0.2,0.04,1.0,1.0,1.0,1.0
